In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import tables

In [ ]:
with tables.open_file('outarray.h5', mode='r') as f:
    tr_loss = np.array(f.root.loss)
    
    tr_hid_1_W = np.array(f.root.hid_1_W)
    tr_hid_1_b = np.array(f.root.hid_1_b)
    tr_hid_1_dW = np.array(f.root.hid_1_dW)
    tr_hid_1_db = np.array(f.root.hid_1_db)
    tr_hid_1_z = np.array(f.root.hid_1_z)
    
    tr_hid_2_W = np.array(f.root.hid_2_W)
    tr_hid_2_b = np.array(f.root.hid_2_b)
    tr_hid_2_dW = np.array(f.root.hid_2_dW)
    tr_hid_2_db = np.array(f.root.hid_2_db)
    tr_hid_2_z = np.array(f.root.hid_2_z)
    
    tr_out_W = np.array(f.root.out_W)
    tr_out_b = np.array(f.root.out_b)
    tr_out_dW = np.array(f.root.out_dW)
    tr_out_db = np.array(f.root.out_db)
    tr_out_z = np.array(f.root.out_z)

In [ ]:
nb_remove = 33
tr_hid_1_z = tr_hid_1_z[:-nb_remove]
tr_hid_2_z = tr_hid_2_z[:-nb_remove]
tr_out_z = tr_out_z[:-nb_remove]

In [ ]:
print('tr_z_hid_1', tr_hid_1_z.shape)
print('tr_z_hid_2', tr_hid_2_z.shape)
print('tr_z_out', tr_out_z.shape)

# Loss

In [ ]:
#
#   Plot Loss, Accuracy
#
fig, ax = plt.subplots(figsize=[12,6])
ax.plot(tr_loss, label='Mini-Batch loss', alpha=.5)

ax.legend()
ax.grid()

plt.tight_layout()
plt.show;

# Weights

In [ ]:
def plot_weights(data_iwn, neuron_nb, title=None, axis=None):
    assert data_iwn.ndim == 3
    assert np.isscalar(neuron_nb)
    
    ni, nw, nn = data_iwn.shape   # iter, weights
    
    if axis is None:
        fig, axis = plt.subplots(1,1, figsize=[16,6])
    
    axis.plot(data_iwn[:,:,neuron_nb], alpha=max(100/nw, 0.002))
    
    if title is not None:
        axis.set_title(title + ' # ' + str(neuron_nb))

In [ ]:
def plot_gradients(data, neuron_nb, title=None, mode='median', color='black', axis=None, figsize=None):
    assert data.ndim == 3
    
    ni, nw, nn = data.shape  # iter, weights, neurons
    
    if neuron_nb == 'all':
        data_iw = data.reshape([ni, -1])
    elif np.issubdtype(type(neuron_nb), np.integer):
        data_iw = data[:,:,neuron_nb]
    else:
        raise ValueError('neuron_nb must be int or "all"')
        
    ratio_abs = np.abs(data_iw)
        
    if axis is None:
        fig, axis = plt.subplots(1,1, figsize=figsize)
        
    if mode == 'full':
        axis.plot(ratio_abs, alpha=max(1/nw, 0.002), color='pink');
        axis.set_title(title + ' raw # ' + str(neuron_nb))

    if mode == 'median':
        axis.plot(np.min(ratio_abs, axis=-1), alpha=.05, color=color);
        axis.plot(np.max(ratio_abs, axis=-1), alpha=.05, color=color);
        axis.plot(np.percentile(ratio_abs, 10, axis=-1), alpha=.3, color=color);
        axis.plot(np.percentile(ratio_abs, 90, axis=-1), alpha=.3, color=color);
        axis.plot(np.median(ratio_abs, axis=-1), alpha=1, color=color);
    
    axis.plot([0,len(ratio_abs)],[.1, .1], ls='--', c='black')
    axis.plot([0,len(ratio_abs)],[.01, .01], ls='-', c='black')
    axis.plot([0,len(ratio_abs)],[.001, .001], ls='--', c='black')
    axis.set_yscale('log')

    if title is not None:
        axis.set_title(title + ' ' + mode + ' # ' + str(neuron_nb))

In [ ]:
def plot_update_ratios(data, neuron_nb, title=None, mode='median', color='red', axis=None, figsize=None):
    assert data.ndim == 3
    
    ni, nw, nn = data.shape  # iter, weights, neurons
    
    if neuron_nb == 'all':
        data_iw = data.reshape([ni, -1])
    elif np.issubdtype(type(neuron_nb), np.integer):
        data_iw = data[:,:,neuron_nb]
    else:
        raise ValueError('neuron_nb must be int or "all"')
        
        
    data_delta_iw = data_iw[1:,:] - data_iw[:-1,:]       # delta
    ratio = data_delta_iw / data_iw[:-1,:]               # delta / weight
    ratio_abs = np.abs(ratio)
    
    if axis is None:
        fig, axis = plt.subplots(1,1, figsize=figsize)
    
    if mode == 'raw':
        axis.plot(data_delta_iw, alpha=max(1/nw, 0.002), color='red');
    else:
        if mode == 'full':
            axis.plot(ratio_abs, alpha=max(1/nw, 0.002), color='pink');
            axis.set_title(title + ' raw # ' + str(neuron_nb))

        if mode == 'median':
            axis.plot(np.min(ratio_abs, axis=-1), alpha=.05, color=color);
            axis.plot(np.max(ratio_abs, axis=-1), alpha=.05, color=color);
            axis.plot(np.percentile(ratio_abs, 10, axis=-1), alpha=.3, color=color);
            axis.plot(np.percentile(ratio_abs, 90, axis=-1), alpha=.3, color=color);
            axis.plot(np.median(ratio_abs, axis=-1), alpha=1, color=color);

        if mode == 'norms':
            data_norm_i = np.linalg.norm(data_iw, axis=-1)
            data_delta_norm_i = np.linalg.norm(data_delta_iw, axis=-1)
            axis.plot(data_delta_norm_i/data_norm_i[:-1], alpha=1, color='green')

        if mode == 'DL4J':
            tmp_ = np.mean(np.abs(data_delta_iw), axis=-1) / np.mean(np.abs(data_iw[:-1,:]), axis=1)
            axis.plot(tmp_, color='blue')

        if mode == 'mean+std':
            mean_ = np.mean(ratio_abs, axis=-1)
            axis.plot(mean_, alpha=1, color='orange');
            std_ = np.std(ratio_abs, axis=-1)
            axis.plot(mean_ + std_, alpha=.3, color='orange');

        axis.plot([0,len(ratio_abs)],[.1, .1], ls='--', c='black')
        axis.plot([0,len(ratio_abs)],[.01, .01], ls='-', c='black')
        axis.plot([0,len(ratio_abs)],[.001, .001], ls='--', c='black')
        axis.set_yscale('log')
        
        if title is not None:
            axis.set_title(title + ' ' + mode + ' # ' + str(neuron_nb))

In [ ]:
# Hidden 1: Individual Neurons
for n in range(3):
    fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=[16,6])
    plot_weights(tr_hid_1_W, neuron_nb=n, title='tr_W_hid_1', axis=ax1)
    plot_update_ratios(tr_hid_1_W, neuron_nb=n, title='tr_W_hid_1', color='red', axis=ax2)
    plot_gradients(tr_hid_1_dW, neuron_nb=n, title='tr_W_hid_1', color='red', axis=ax3)
    fig.tight_layout()

In [ ]:
# Hidden 1: All Biases
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=[16,6])
plot_weights(tr_hid_1_b[:,:,np.newaxis], neuron_nb=0, title='tr_hid_1_b', axis=ax1)
plot_update_ratios(tr_hid_1_b[:,:,np.newaxis], neuron_nb=0, title='tr_hid_1_b', color='red', axis=ax2)
plot_gradients(tr_hid_1_db[:,:,np.newaxis], neuron_nb=0, title='tr_hid_1_db', color='red', axis=ax3)
fig.tight_layout()

In [ ]:
# Hidden 1: Summary
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=[16,5])
plot_update_ratios(tr_hid_1_W, neuron_nb='all', title='tr_hid_1_W', color='red', axis=ax1)
plot_gradients(tr_hid_1_dW, neuron_nb='all', title='tr_hid_1_dW', color='red', axis=ax2)
fig.tight_layout()

In [ ]:
# Hidden 2: Individual Neurons
for n in range(3):
    fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=[16,6])
    plot_weights(tr_hid_2_W, neuron_nb=n, title='tr_hid_2_W', axis=ax1)
    plot_update_ratios(tr_hid_2_W, neuron_nb=n, title='tr_hid_2_W', color='green', axis=ax2)
    plot_gradients(tr_hid_2_dW, neuron_nb=n, title='tr_hid_2_dW', color='green', axis=ax3)
    fig.tight_layout()

In [ ]:
# Hidden 2: All Biases
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=[16,6])
plot_weights(tr_hid_2_b[:,:,np.newaxis], neuron_nb=0, title='tr_hid_2_b', axis=ax1)
plot_update_ratios(tr_hid_2_b[:,:,np.newaxis]+1e-6, neuron_nb=0, title='tr_hid_2_b', color='green', axis=ax2)
plot_gradients(tr_hid_2_db[:,:,np.newaxis], neuron_nb=0, title='tr_hid_2_db', color='green', axis=ax3)
fig.tight_layout()

In [ ]:
# Hidden 2: Summary
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=[16,5])
plot_update_ratios(tr_hid_2_W, neuron_nb='all', title='tr_hid_2_W', color='green', axis=ax1)
plot_gradients(tr_hid_2_dW, neuron_nb='all', title='tr_hid_2_dW', color='green', axis=ax2)
fig.tight_layout()

In [ ]:
# Output: Individual Neurons
for n in range(3):
    fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=[16,6])
    plot_weights(tr_out_W, neuron_nb=n, title='tr_out_W', axis=ax1)
    plot_update_ratios(tr_out_W, neuron_nb=n, title='tr_out_W', color='blue', axis=ax2)
    plot_gradients(tr_out_dW, neuron_nb=n, title='tr_out_dW', color='blue', axis=ax3)
    fig.tight_layout()

In [ ]:
# Output: All Biases
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=[16,6])
plot_weights(tr_out_b[:,:,np.newaxis], neuron_nb=0, title='tr_out_b', axis=ax1)
plot_update_ratios(tr_out_b[:,:,np.newaxis]+1e-6, neuron_nb=0, title='tr_out_b', color='blue', axis=ax2)
plot_gradients(tr_out_db[:,:,np.newaxis], neuron_nb=0, title='tr_out_db', color='blue', axis=ax3)
fig.tight_layout()

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=[16,5])
plot_update_ratios(tr_out_W, neuron_nb='all', title='tr_out_W', color='blue', axis=ax1)
plot_gradients(tr_out_dW, neuron_nb='all', title='tr_out_dW', color='blue', axis=ax2)
fig.tight_layout()

# Activations

In [ ]:
def plot_3d_histogram(data, es, neuron_nb, funct=None, 
                      skip_first=False, lines_01=True, title=None, color=(1,0,0,1), ax=None, figsize=None):
    assert data.ndim == 3
    
    if funct is None:
        funct = lambda x: x
        
    if isinstance(funct, str):
        if funct == 'sigmoid': funct = lambda x: 1 / (1 + np.exp(-x))
        elif funct == 'tanh': funct = lambda x: np.tanh(x)
        elif funct == 'softsign': funct = lambda x: x / (1+np.abs(x))
        elif funct == 'relu': funct = lambda x: np.maximum(0, x)
        elif funct == 'lrelu': funct = lambda x: np.where(x > 0, x, x * 0.01)
        else: raise ValueError('Unknown function string')
    
    ni, na, nn = data.shape  # iter, activations (batch size), neurons
    
    if neuron_nb == 'all':
        #data_ia = data.reshape([ni, -1])
        data_ia = data.reshape([-1, es*nn])
    elif np.issubdtype(type(neuron_nb), np.integer):
        #data_ia = data[:,:,neuron_nb]
        data_ia = data[:,:,neuron_nb].reshape(-1, es)
    else:
        raise ValueError('neuron_nb must be int or "all"')
        
    
    
    def interpolate_colors(cstart, cend, n):
        cstart, cend = np.array(cstart), np.array(cend)
        assert cstart.shape == (4,)
        assert cend.shape == (4,)
        if n == 1:  return cend    # if one step, then return end color

        cols = []
        for i in range(n):
            step = i/(n-1)
            cols.append( (1-step)*cstart + step*cend)
        return np.array(cols)
    
    color = np.array(color)
    color_start = np.array(color/4, dtype=float)  # transparent black
    color_end = np.array(color)
    colors = interpolate_colors(color_start, color_end, len(data_ia))
    

    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, projection='3d')
    
    
    ax.view_init(30, -85)
    
    for epoch in range(len(data_ia)):                                     # One line per epoch
        hist, bins = np.histogram(funct(data_ia[epoch,:]), bins=100)      # apply funct and create histogram
        bins = (bins[:-1] + bins[1:])/2                                   # center bins
        hist = hist / np.sum(hist)
        
        hist_0 = hist[0]
        if skip_first:
            bins = bins[1:]
            hist = hist[1:]
        
        ax.plot(xs=bins, ys=hist,
                zs=-epoch,
                zdir='y', 
                color=colors[epoch])
        nb_epochs = len(data_ia)
        if epoch == 0 and lines_01:
            ax.plot(xs=[0,0], ys=[0,0], zs=[-nb_epochs,0], zdir='y', color='k')
            ax.plot(xs=[1,1], ys=[0,0], zs=[-nb_epochs,0], zdir='y', color='k', ls='--')
        if epoch == len(data_ia)-1:
            ax.plot(xs=[bins[0],bins[-1]], ys=[0,0], zs=-nb_epochs, zdir='y', color='k')
    
    if skip_first:
        ax.set_xlabel('value ('+str(round((1-hist_0)*100, 2)) + '%)'); ax.set_ylabel('epoch'); ax.set_zlabel('n')
    else:
        ax.set_xlabel('value'); ax.set_ylabel('epoch'); ax.set_zlabel('n')
        
    if title is not None:
        ax.set_title(title)

In [ ]:
es = 100000  # epoch_size
act_fun = 'relu'

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, subplot_kw=dict(projection='3d'), figsize=[16,5])
plot_3d_histogram(tr_hid_1_z, es=es, neuron_nb='all', title='Hidden 1 raw', funct=None, color=(0,0,0,1), ax=ax1)
plot_3d_histogram(tr_hid_1_z, es=es, neuron_nb='all', title='Hidden 1', skip_first=True, funct=act_fun, color=(1,0,0,1), ax=ax2)
fig.tight_layout()

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, subplot_kw=dict(projection='3d'), figsize=[16,5])
plot_3d_histogram(tr_hid_2_z, es=es, neuron_nb='all', title='Hidden 2 raw', funct=None, color=(0,0,0,1), ax=ax1)
plot_3d_histogram(tr_hid_2_z, es=es, neuron_nb='all', title='Hidden 2', skip_first=True, funct=act_fun, color=(0,1,0,1), ax=ax2)
fig.tight_layout()

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, subplot_kw=dict(projection='3d'), figsize=[16,5])
plot_3d_histogram(tr_out_z, es=es, neuron_nb='all', title='Output raw', funct=None, color=(0,0,0,1), ax=ax1)
plot_3d_histogram(tr_out_z, es=es, neuron_nb='all', title='Output', funct=None, color=(0,0,1,1), ax=ax2)
fig.tight_layout()

In [ ]:
#
#   Hidden 1 NEURONS ACTIVATIONS
#
plot_n = 10  # tr_z_hid.shape[-1]
fig, axes = plt.subplots(nrows=plot_n//2, ncols=4, subplot_kw=dict(projection='3d'), figsize=[16,2*plot_n/2])

for n in range(plot_n):
    plot_3d_histogram(tr_hid_1_z, es=es, neuron_nb=n, title='Neuron #'+str(n), funct=None, color=(0,0,0,1), ax=axes[n//2][2*(n%2)])
    plot_3d_histogram(tr_hid_1_z, es=es, neuron_nb=n, title='Neuron #'+str(n), funct=act_fun, skip_first=True, color=(1,0,0,1), ax=axes[n//2][2*(n%2)+1])

fig.tight_layout()

In [ ]:
#
#   Hidden 2 NEURONS ACTIVATIONS
#
plot_n = 10  # tr_z_hid.shape[-1]
fig, axes = plt.subplots(nrows=plot_n//2, ncols=4, subplot_kw=dict(projection='3d'), figsize=[16,2*plot_n/2])

for n in range(plot_n):
    plot_3d_histogram(tr_hid_2_z, es=es, neuron_nb=n, title='Neuron #'+str(n), funct=None, color=(0,0,0,1), ax=axes[n//2][2*(n%2)])
    plot_3d_histogram(tr_hid_2_z, es=es, neuron_nb=n, title='Neuron #'+str(n), funct=act_fun, skip_first=True, color=(0,1,0,1), ax=axes[n//2][2*(n%2)+1])

fig.tight_layout()

In [ ]:
#
#   Output NEURONS ACTIVATIONS
#
plot_n = 4  # tr_z_out.shape[-1]
fig, axes = plt.subplots(nrows=plot_n//2, ncols=4, subplot_kw=dict(projection='3d'), figsize=[16,2*plot_n/2])

for n in range(plot_n):
    
    if n >= 3:
        break
    
    plot_3d_histogram(tr_out_z, es=es, neuron_nb=n, title='Neuron #'+str(n), funct=None, color=(0,0,0,1), ax=axes[n//2][2*(n%2)])
    plot_3d_histogram(tr_out_z, es=es, neuron_nb=n, title='Neuron #'+str(n), funct=None, color=(0,0,1,1), ax=axes[n//2][2*(n%2)+1])
    
fig.tight_layout()